In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import DataFrame
spark

In [ ]:
%run ../ssb_sparktools/quality/quality.py

In [ ]:
%run ../ssb_sparktools/processing/processing.py

In [ ]:
READ_PATH = '/produkt/skatt/person/temp/etablere_klargjoring/skatteobjekt/'

In [ ]:
aksjesparekonto = spark.read.path(READ_PATH+'aksjesparekonto')

In [ ]:
ask_w_ds = spark_qual_missing(aksjesparekonto, 'Aksjesparekonto')

In [ ]:
ask = spark_qual_missing(aksjesparekonto)

In [ ]:
ask_w_ds.toPandas()

In [ ]:
ask.toPandas()

In [ ]:
minste = spark.read.path(READ_PATH+'minstefradragOgKostnader')

In [ ]:
testminste = spark_qual_missing(minste)

In [ ]:
testminste.toPandas()

In [ ]:
minste.toPandas()

In [ ]:
tverrsnitt_skattemelding = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding')

In [ ]:
so_dict = unpack_parquet(tverrsnitt_skattemelding, 
                      rootdf=True, 
                      rootvar=['personidentifikator']
                     )

In [ ]:
%%time
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

missing_variabler = [StructField('dataframe', StringType(), False),\
                   StructField('variable', StringType(), True),\
                   StructField('no_missing', IntegerType(), False)]
missing_schema = StructType(missing_variabler)
df_missing = sqlContext.createDataFrame(sc.emptyRDD(), missing_schema)

for k in so_dict.keys():
    missing_so = spark_qual_missing(so_dict[k], k)
    df_missing = df_missing.union(missing_so)

In [ ]:
df_missing.toPandas()

In [ ]:
spark.read.path('/produkt/skatt/person/temp/missingcorrection_log')

In [ ]:
corrected.filter(corrected['dataframe']=='virtuellValuta').toPandas()